Exercises created by Qwen2.5-Max

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import time

Manipulação de Tensores  

1.1 Altere o código para carregar um conjunto de dados diferente, como o CIFAR-10 (torchvision.datasets.CIFAR10). <br>
1.2 Modifique o tamanho do lote (batch_size) para 32 e observe como isso afeta o tempo de treinamento. <br>
1.3 Adicione uma transformação personalizada ao pipeline de dados, como normalizar os valores dos pixels para o intervalo [-1, 1]. <br>
     

In [2]:
# ToTensor transforms from (0, 255) to (0, 1), but i want to transform it to (-1, 1),
# which is better to work with activation functions such as Tahn, while sigmoid is better
# with (0, 1).

# this refers to the value a pixel can take, where (0, 255) represents a normal representation
# most images format, where 0 is black and 255 white.

# creating a transformation function

transform = transforms.Compose([
    transforms.ToTensor(), # normalise to [0, 1]
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # normalise to [-1, 1]
])

# Download training data from open datasets.
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=transform
)

# Download test data from open datasets.
test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=transform
)

# modifying the batch size to to 32 e compare o tempo de treino com 64

batch_size_64 = 64
batch_size_32 = 32

# dataloaders
train_dataloader_64 = DataLoader(training_data, batch_size=batch_size_64)
train_dataloader_32 = DataLoader(training_data, batch_size=batch_size_32)

# find out what the shape necessary for the input
for X, y in train_dataloader_64:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

100.0%


Shape of X [N, C, H, W]: torch.Size([64, 3, 32, 32])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
# making sure it runs on gpu
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

# defining model
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3*32*32, 512),
            nn.Tanh(),
            nn.Linear(512, 512),
            nn.Tanh(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = SimpleNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# train loop
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Computer prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

epochs = 5

# training loop with batch_size 64
start_test1 = time.time() # save initial time
for t in range(epochs):
    print(f"Epoch dataloader_64 {t+1}\n--------------")
    train(train_dataloader_64, model, loss_fn, optimizer)
end_test1 = time.time() # save end time

# training loop with batch_size 32
start_test2 = time.time()
for t in range(epochs):
    print(f"Epoch dataloader_32 {t+1}\n--------------")
    train(train_dataloader_32, model, loss_fn, optimizer)
end_test2 = time.time()

print(f"Time elapsed for batch_size of 64:{end_test1-start_test1:5f}")
print(f"Time elapsed for batch_size of 32:{end_test2-start_test2:5f}")

Epoch dataloader_64 1
--------------
loss: 2.304279 [   64/50000]
loss: 2.247720 [ 6464/50000]
loss: 2.209546 [12864/50000]
loss: 2.207372 [19264/50000]
loss: 2.126846 [25664/50000]
loss: 2.129681 [32064/50000]
loss: 2.141927 [38464/50000]
loss: 2.054298 [44864/50000]
Epoch dataloader_64 2
--------------
loss: 2.117157 [   64/50000]
loss: 2.088054 [ 6464/50000]
loss: 1.998731 [12864/50000]
loss: 2.090847 [19264/50000]
loss: 2.015155 [25664/50000]
loss: 2.025780 [32064/50000]
loss: 2.085927 [38464/50000]
loss: 1.950084 [44864/50000]
Epoch dataloader_64 3
--------------
loss: 2.048022 [   64/50000]
loss: 2.014682 [ 6464/50000]
loss: 1.869377 [12864/50000]
loss: 2.016907 [19264/50000]
loss: 1.979053 [25664/50000]
loss: 1.973342 [32064/50000]
loss: 2.050645 [38464/50000]
loss: 1.895323 [44864/50000]
Epoch dataloader_64 4
--------------
loss: 1.996940 [   64/50000]
loss: 1.971885 [ 6464/50000]
loss: 1.782338 [12864/50000]
loss: 1.962134 [19264/50000]
loss: 1.957988 [25664/50000]
loss: 1.943

Definição do Modelo  

2.1 Altere o modelo para incluir mais uma camada oculta com 128 neurônios e uma função de ativação ReLU. <br>
2.2 Substitua a função de ativação ReLU por outra, como LeakyReLU ou Tanh. Compare os resultados. <br>
2.3 Implemente um modelo usando uma arquitetura convolucional (CNN) em vez de uma rede totalmente conectada. Use pelo menos duas camadas convolucionais seguidas por camadas de pooling. <br>

Treinamento do Modelo  

3.1 Altere o número de épocas para 10 e observe como isso afeta a precisão do modelo.<br>
3.2 Adicione um mecanismo de parada antecipada (early stopping) que interrompe o treinamento se a perda de validação não melhorar por 3 épocas consecutivas. <br>
3.3 Experimente diferentes funções de perda, como nn.CrossEntropyLoss e nn.NLLLoss. Explique as diferenças entre elas e como elas afetam o treinamento. <br>
3.4 Implemente um otimizador diferente, como Adam (torch.optim.Adam), e compare seu desempenho com o SGD. <br>

Avaliação do Modelo  

4.1 Calcule outras métricas de avaliação, como recall, precisão e F1-score, para avaliar o desempenho do modelo. <br>
4.2 Plote a matriz de confusão para visualizar os erros do modelo. <br>
4.3 Salve o modelo treinado usando torch.save e carregue-o novamente usando torch.load. <br>
4.4 Teste o modelo carregado em novos dados. <br>

Experimentação e Análise  

5.1 Realize experimentos com diferentes taxas de aprendizado (learning rate). Qual é a taxa ideal para o problema? <br>
5.2 Aplique regularização L2 (weight decay) ao otimizador e observe como isso afeta o desempenho do modelo. <br>
5.3 Use dropout (nn.Dropout) no modelo para reduzir o overfitting. Teste diferentes valores de probabilidade de dropout (ex.: 0.2, 0.5). <br>
5.4 Experimente diferentes inicializações de pesos para as camadas do modelo (ex.: torch.nn.init.xavier_uniform_). <br>

Desafio Final  

Combine tudo o que aprendeu para resolver um problema mais complexo. 

1.1 Escolha um novo conjunto de dados (por exemplo, MNIST, CIFAR-10 ou qualquer outro disponível no torchvision.datasets). <br>
1.2 Projete e implemente um modelo CNN personalizado para classificar as imagens desse conjunto de dados. <br>
1.3 Treine o modelo, ajuste os hiperparâmetros e avalie seu desempenho. <br>
1.4 Documente suas descobertas e explique quais decisões de design contribuíram para o sucesso do modelo. <br>
     